# Automated Term Extraction

In [52]:
from pyate import combo_basic
import googletrans
import feedparser
from urllib.request import urlopen
from bs4 import BeautifulSoup



def get_only_text(url):
    
    """
    return the title and the text of the article at the 
    specified url
    """
    
    page=urlopen(url)
    soup=BeautifulSoup(page,"html.parser")
    text=' '.join(map(lambda p: p.text, soup.find_all('p')))
    # Eliminar Última Hora
    rest = soup.find_all('p', attrs={'class' : "line-clamp_x2 ue-c-widget__article-headline"})[0].text
    index = text.find(rest)
    final_text = text[:index]
    return [final_text,soup.title.text]

In [53]:
url_expansion = 'https://e00-expansion.uecdn.es/rss/empresasbanca.xml'
expansion = feedparser.parse(url_expansion)
expansion_entries = expansion['entries']

print('Total Entradas Expansión:',len(expansion_entries))

Total Entradas Expansión: 35


In [74]:
string_es = get_only_text(expansion_entries[1]['id'])[0]

In [75]:
string_es

'Bankia ha incorporado a Raimundo Sala, hasta ahora primer ejecutivo de PayPal para Desarrollo de Negocio Internacional, con el fin de reforzar el proceso de transformación digital que está llevando a cabo la entidad desde hace ya algunos años, como adelantó EXPANSIÓN.  Sala ha sido nombrado director corporativo de Soluciones de Negocio de Bankia, una dirección de nueva creación y que será clave para el desarrollo de los medios de pago en el ecosistema digital, además de los nuevos negocios relacionados con el mundo online. Reportará directamente a la Dirección General Adjunta de Banca de Particulares, a cuyo frente está Eugenio Solla. "Es para mi todo un reto profesional contribuir a la transformación digital Bankia desde la Dirección de Soluciones de Negocio. Estoy seguro de que vamos a ofrecer la mejor experiencia de cliente en el ámbito de los pagos digitales, que actualmente está experimentando una rápida y profunda evolución y que es fundamental para cumplir nuestro compromiso de

In [78]:
string = googletrans.Translator().translate(string_es,dest='en').text
print(combo_basic(string).sort_values(ascending=False).head(5))

international business development    2.397225
international business                2.136294
business development                  2.136294
digital transformation                2.136294
business phone                        1.443147
dtype: float64


In [77]:
import spacy
from pyate.term_extraction_pipeline import TermExtractionPipeline

nlp = spacy.load("en_core_web_sm")
nlp.add_pipe(TermExtractionPipeline())
doc = nlp(string)
print('### ENG ###')
print(doc._.combo_basic.sort_values(ascending=False).head(5))
doc._.combo_basic.index = [googletrans.Translator().translate(i,dest='es').text for i in doc._.combo_basic.index]
print('### ESP ###')
print(doc._.combo_basic.sort_values(ascending=False).head(5))

### ENG ###
digital transformation    2.136294
phone numbers             1.443147
transformation process    1.443147
digital payment           1.443147
payment gateways          1.443147
dtype: float64
### ESP ###
transformación digital       2.136294
números de teléfono          1.443147
Proceso de transformación    1.443147
pago digital                 1.443147
Via de pago                  1.443147
dtype: float64
